### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

# Описание данных

train - обучающая


test - по которой строится лидерборд


*.pckl - это полные файлы (запикленные датафреймы) train_sample.pckl.zip - первые 100000 строк трейна (запикленные датафреймы)


*.pckl - это pickle-файлы при открытии которых открываются в датафреймы


вас может удивить большое число данных, это сделано специально в реальном мире вам доступна куча данных, потому вам всегда надо решать какие из них важны а какие нет трейн и тест фактически продублированы CSV файлами в которых есть только часть столбцов (чтоб можно было считать кусочками) так же есть первые 100000 строк трейна запикленных отдельно


использовать все данные не всегда обязательно, не исключено наличие мусора который вам не понадобится

train_4_col_2.csv.zip - кусок TRAIN, который содержит id, category, subcategory (строки все)

test_3_col_2.csv.zip - кусок TEST, который содержит id, category, subcategory (строки все)

train_4_col_3.csv.zip - кусок TRAIN, который содержит id, fields (строки все)

test_3_col_3.csv.zip - кусок TEST, который содержит id, fields (строки все)

train_sample.pckl.zip - кусок TRAIN, который содержит все поля, но только первые 100000 строк

train_4_col.csv.zip - кусок TRAIN, который содержит id, name, description, price (строки все)

test_4_col.csv.zip - кусок TEST, который содержит id, name, description (строки все)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
train_cat_subcat = pd.read_csv('./hack/train_4_col_2.csv', sep='\t').set_index('Unnamed: 0')

In [3]:
test_cat_subcat = pd.read_csv('./hack/test_3_col_2.csv', sep='\t').set_index('Unnamed: 0')

In [4]:
train_desc_price = pd.read_csv('./hack/train_4_col.csv', sep='\t').set_index('Unnamed: 0')

In [5]:
train_cat_subcat = train_cat_subcat.merge(train_desc_price, on='id')
train_cat_subcat.head()

,id,category,subcategory,name,description,price
0,3edeb34cf93f490ff760af85,9,914,Сумка DG,NaN,199900
1,c98febd50dad3cc0ffc86085,22,2202,Комплект,8-12 лет,35000
2,ade01e13912a46a99134cc75,22,2204,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000
3,ab3e6941c11304c1519aef75,22,2209,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000
4,252452a91c944a22c276d995,22,2204,Жилет теплый,NaN,50000


In [6]:
targets = train_cat_subcat['price'].values[:1000000]

In [7]:
targets.shape

(1000000,)

In [8]:
list_pickle_path = 'clean_train_vacs.pkl'
with open(list_pickle_path, 'rb') as f:
    clean_train_vacs = pickle.load(f)
f.close()

In [10]:
%%time
print ('Фигачим Bag of words...\n')
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 10000) 


# 5 000 фич считались 20.1 сек
# прогоняем полученный в прошлом пункте лист с текстами через fit_transform() 
train_data_features = vectorizer.fit_transform(clean_train_vacs[:1000000])

# Переделаем полученные свойства в array 
train_data_features = train_data_features.toarray()

Фигачим Bag of words...

CPU times: user 17.2 s, sys: 11.7 s, total: 28.8 s
Wall time: 30.1 s


In [11]:
train_data_features.shape

(1000000, 10000)

In [12]:
train_cat_subcat2 = pd.read_csv('./hack/train_4_col_2.csv', sep='\t').set_index('Unnamed: 0')


In [13]:
print(train_cat_subcat2.shape)
print(test_cat_subcat.shape)

(1748890, 3)
(749525, 3)


In [14]:
test_cat_subcat.head()

,id,category,subcategory
Unnamed: 0,,,
4,285ea2e9935ccdeb8378c6a5,6,603
9,adfb73820bbb831257df6e95,2,203
15,783025601c36202f633fc6a5,1,116
19,2f0cd2d2e15dc90afd847f95,10,1009
20,5c23a37902855a20172845a5,11,1104


In [15]:
train_cat_subcat2['sample'] = 'train'
test_cat_subcat['sample'] = 'test'

In [16]:
dft = pd.concat([train_cat_subcat2, test_cat_subcat], axis = 0)
dft.head()

,id,category,subcategory,sample
Unnamed: 0,,,,
0,3edeb34cf93f490ff760af85,9,914,train
1,c98febd50dad3cc0ffc86085,22,2202,train
2,ade01e13912a46a99134cc75,22,2204,train
3,ab3e6941c11304c1519aef75,22,2209,train
5,252452a91c944a22c276d995,22,2204,train


In [17]:
dft.shape

(2498415, 4)

In [18]:
ohe = pd.get_dummies(dft.drop('id', axis = 1), columns=['category','subcategory'])

In [19]:
ohe.head()

,sample,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,...,subcategory_2210,subcategory_2211,subcategory_2212,subcategory_2213,subcategory_2214,subcategory_2215,subcategory_2216,subcategory_2301,subcategory_10001,subcategory_10002
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,train,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,train,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,train,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,train,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,train,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
ohe_train = ohe[ohe['sample'] == 'train'].drop('sample', axis = 1).values[:1000000]
ohe_test = ohe[ohe['sample'] == 'test'].drop('sample', axis = 1).values
print(ohe_train.shape)
print(ohe_test.shape)

(1000000, 274)
(749525, 274)


In [21]:
train_data_features = np.column_stack((train_data_features, ohe_train))
print(train_data_features.shape)

(1000000, 10274)


In [22]:
del ohe_train

In [23]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
def build_model(): 
    """
    так как нам потребуется несколько экземпляров одной и той же модели,
    мы определили функцию для её создания
    """
    model = models.Sequential()
    model.add(layers.Dense(512, activation='relu', input_shape=(train_data_features.shape[1],)))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1024, activation='relu'))    
    model.add(layers.Dense(1))
    model.compile(optimizer=RMSprop(lr=5e-2), loss='mean_squared_logarithmic_error', metrics=['mean_squared_logarithmic_error'])
    return model

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
model = build_model()
model.fit(train_data_features, targets, epochs = 20, batch_size = 256, verbose = 1)

Epoch 1/20
1000000/1000000 [==============================] - 446s 446us/step - loss: 2.7292 - mean_squared_logarithmic_error: 2.7292
Epoch 2/20
1000000/1000000 [==============================] - 405s 405us/step - loss: 2.4187 - mean_squared_logarithmic_error: 2.4187
Epoch 3/20
1000000/1000000 [==============================] - 407s 407us/step - loss: 2.3436 - mean_squared_logarithmic_error: 2.3436
Epoch 4/20
1000000/1000000 [==============================] - 414s 414us/step - loss: 2.2594 - mean_squared_logarithmic_error: 2.2594
Epoch 5/20
1000000/1000000 [==============================] - 404s 404us/step - loss: 2.1755 - mean_squared_logarithmic_error: 2.1755
Epoch 6/20
1000000/1000000 [==============================] - 422s 422us/step - loss: 2.1398 - mean_squared_logarithmic_error: 2.1398
Epoch 7/20
1000000/1000000 [==============================] - 413s 413us/step - loss: 2.0743 - mean_squared_logarithmic_error: 2.0743
Epoch 8/20
1000000/1000000 [==============================] - 

In [25]:
del targets
del train_data_features
del clean_train_vacs

In [26]:
list_pickle_path = '20_epochs_wide_model_lr=0.05_1000000.pkl'
list_pickle = open(list_pickle_path, 'wb')
pickle.dump(model, list_pickle)
list_pickle.close()

In [27]:
%%time
list_pickle_path = 'clean_test_vacs.pkl'
with open(list_pickle_path, 'rb') as f:
    clean_test_vacs = pickle.load(f)
f.close()

test_data_features = vectorizer.transform(clean_test_vacs)
test_data_features = test_data_features.toarray()
print(test_data_features.shape)

(749525, 10000)
CPU times: user 15.4 s, sys: 6.94 s, total: 22.3 s
Wall time: 22.6 s


In [28]:
del clean_test_vacs

In [29]:
test_data_features = np.column_stack((test_data_features, ohe_test))
print(test_data_features.shape)

(749525, 10274)


In [30]:
y_pred_wide_lr005_1000000 = model.predict(test_data_features, verbose=1)

749525/749525 [==============================] - 159s 212us/step


In [31]:
list_pickle_path = 'y_pred_wide_lr005.pkl'
list_pickle = open(list_pickle_path, 'wb')
pickle.dump(y_pred_wide_lr005_1000000, list_pickle)
list_pickle.close()

In [32]:
import pandas as pd
subsam = pd.read_csv('./hack/submit_Sample.csv', sep=',')
subsam['price'] = [0 if x[0]<0 else x[0] for x in y_pred_wide_lr005_1000000]
subsam.head()

,id,price
0,285ea2e9935ccdeb8378c6a5,1.090320e+05
1,adfb73820bbb831257df6e95,3.808328e+04
2,783025601c36202f633fc6a5,5.150555e+06
3,2f0cd2d2e15dc90afd847f95,6.328730e+04
4,5c23a37902855a20172845a5,9.628400e+04


In [33]:
subsam.to_csv( "y_pred_wide_lr005_1000000.csv", sep=',', encoding='utf-8',index=False )